### 코랩 서버 스펙 확인

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17912622796342004568
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14442440736241332949
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [4]:
import platform
platform.platform()

'Linux-5.10.147+-x86_64-with-glibc2.31'

## __데이터셋 불러오기__

In [13]:
import pandas as pd
train_set = pd.read_csv('/content/drive/MyDrive/Github/floread/sentiment-analysis/data/감성대화말뭉치(최종데이터)_Training.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Github/floread/sentiment-analysis/data/감성대화말뭉치(최종데이터)_Validation.csv')
train_set.sample(n=5)

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
44836,44837,청년,남성,대인관계,해당없음,분노,툴툴대는,나는 취업할 의지도 없는 백수인 거 같아 이런 내가 실망스러워.,자기 자신한테 실망하셨군요. 안쓰럽네요.,내가 뭐가 되고 싶었는지도 모르겠어.,하고 싶은 일이 없으시군요. 뭘 해야 할까요?,남들이 일하는 모습을 봐야 뭐라도 되고 싶지 않을까?,남들이 일하는 모습을 보는 것도 좋은 방법이네요. 꿈이 생기길 바라요.
5942,5943,중년,여성,"재정,은퇴,노후준비",해당없음,불안,당혹스러운,해외에 나간 적도 없는데 해외에서 결제가 됐다는 당혹스러운 문자를 받았어.,하지도 않은 해외 결제 문자가 와서 정말 당황스러웠겠어요.,카드사에 문의했더니 피싱 사기 같대. 문제 생길까 봐 걱정돼.,갑작스러운 피싱 사기 염려로 매우 불안하실 것 같아요.,NaN,NaN
27839,27840,중년,여성,건강,해당없음,당황,열등감,돈이라도 벌어야 내가 아픈 곳을 치료할 텐데 지금 나의 형편이 너무 좋지 않아서 괴로워.,아픈 곳을 치료받지 못한 것 때문에 마음이 괴로우셨군요.,이렇게 형편이 안 좋을 바에는 차라리 아프고 마는 게 더 나을 것 같아.,어떻게 하면 지금 상황이 나아질 수 있을까요?,모르겠어. 나는 아무것도 할 수 없는 상태야.,주위에 도움을 받을 수 있는 사람이 있었으면 좋겠어요.
39561,39562,노년,남성,재정,만성질환 무,상처,"가난한, 불우한",정년이 되어서 퇴직을 하게 되니 자식들과 의견 충돌이 자주 있어 괴로워.,자식들과의 의견 차이로 힘들어하시는 것 같아요.,퇴직 후 시간적 여유가 있게 되어 다른 일을 해보려고 하니 자식들이 못하게 해.,왜 그렇게 생각하시나요?,자식들의 생각과 나의 생각이 전혀 달라서 힘들어.,퇴직 후 자식들과 의견 차이로 힘드시군요.
17701,17702,중년,남성,"대인관계(부부, 자녀)",해당없음,기쁨,흥분,우리 아내 미용실 오픈이 바로 오늘이야! 회사까지 그만두고 하고 싶다던 일에 뛰어들었지.,아내 분께서 회사까지 그만두고 하고 싶었던 미용실을 오픈 하시는군요.,처음 들었을 땐 걱정했지만 노력해서 미용사 자격증을 따는 모습을 보고 나도 응원해줬어.,열심히 공부해서 미용사 자격증을 따시다니 정말 자랑스러웠겠어요.,정말 자랑스러워. 이제 미용실 개업도 하니까 나도 가서 거들어 줘야겠어.,미용실 개업이 잘 되었으면 좋겠어요.


In [9]:
# # 감정_대분류, 문장 외에 칼럼은 드랍(나중엔 쓰일 수 있음)
# train_set = train_set.drop(columns = ["Unnamed: 0","연령","성별","상황키워드","신체질환","감정_소분류"])
# train_set.head()

,감정_대분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


> __데이터설명서:__
첫 번째 발화에는 이러한 감정이 드러나도록 문장을 생성하는데, 첫 문장 발화에 대한 시스템 응답 및 사용자의 자연스러운 후속 대화에서는 상황에 따라 감정 상태가 포함되지 않은 문장이 출현할 수 있다.

 **우선 사람문장1 - 감정대분류만 활용**


In [14]:
train_set = train_set[['감정_대분류','사람문장1']]
test_set = test_set[['감정_대분류','사람문장1']]

train_set.columns = ['emotion', 'sentence']  # 칼럼이름 변경
test_set.columns = ['emotion', 'sentence']  # 칼럼이름 변경

train_set.head()

,emotion,sentence
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.


In [15]:
# 감정을 정수 라벨로 변경
emotions = {'기쁨': 0, '불안': 1, '당황': 2, '슬픔': 3, '분노': 4, '상처': 5}

train_set['emotion'] = train_set.emotion.map(emotions)
test_set['emotion'] = test_set.emotion.map(emotions)

train_set.sample(n=5)
# train_set.loc[(train_set['emotion'] == "기쁨"), 'emotion'] = 0  #기쁨 => 0
# train_set.loc[(train_set['emotion'] == "불안"), 'emotion'] = 1  #불안 => 1
# train_set.loc[(train_set['emotion'] == "당황"), 'emotion'] = 2  #당황 => 2
# train_set.loc[(train_set['emotion'] == "슬픔"), 'emotion'] = 3  #슬픔 => 3
# train_set.loc[(train_set['emotion'] == "분노"), 'emotion'] = 4  #분노 => 4
# train_set.loc[(train_set['emotion'] == "상처"), 'emotion'] = 5  #상처 => 5

,emotion,sentence
35320,0,난 건강한 요리를 맛있게 할 수 있어. 자식들이 잘 먹어줘서 기분이 좋아.
42733,2,엄마가 학교생활은 어떠냐고 물으셨는데 짜증을 내고 말았어. 엄마한테 너무 죄송해.
27018,0,요즘 내 주변 사람들이 날 부러워 해.
6958,1,내가 앞으로 잘해나갈 수 있을지 너무 걱정돼.
42409,4,엄마와 이번 시험에 평균을 올리기로 약속했는데 올리지 못했어. 나 자신이 실망스러워.


__Korean BERT pre-trained cased (KoBERT) for Huggingface Transformers__
<br>
koBERT Hugging Face 토크나이저 및  Requirements 설치


In [16]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install sentencepiece
!pip install transformers
!pip install torch

!pip install mxnet
!pip install gluonnlp pandas tqdm   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hurwxqqn/kobert-tokenizer_c574e7fcc5b84f87b26f20a543c8909f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hurwxqqn/kobert-tokenizer_c574e7fcc5b84f87b26f20a543c8909f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [17]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
 
#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = torch.device("cuda:0")

__토크나이저, pretrained 모델, vocabulary 로드__

In [18]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters(KoBERT finetuning 베에스 라인을 그대로 따랐음)
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [19]:
# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [20]:
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, #클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
# 정확도 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
# 입력에 대한 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [24]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]
test_set_data = [[i, str(j)] for i, j in zip(test_set['sentence'], test_set['emotion'])]

# # sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
# train_set_data, valid_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)
# 이번엔 데이터 셋에서 제공되는 테스트 셋 사용했음

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

In [25]:
# 학습
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-25-b9384cb2990c>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/807 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7632750272750854 train acc 0.234375
epoch 1 batch id 201 loss 1.337585687637329 train acc 0.2922885572139303
epoch 1 batch id 401 loss 0.8853484392166138 train acc 0.43504519950124687
epoch 1 batch id 601 loss 1.7470868825912476 train acc 0.473585690515807
epoch 1 batch id 801 loss 0.7932302355766296 train acc 0.47054463171036204
epoch 1 train acc 0.47192954986261515


<ipython-input-25-b9384cb2990c>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정


  0%|          | 0/104 [00:00<?, ?it/s]

...

epoch 4 test acc 0.46346399136577704


  0%|          | 0/807 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.128096342086792 train acc 0.65625
epoch 5 batch id 201 loss 0.8485383987426758 train acc 0.6948849502487562
epoch 5 batch id 401 loss 0.46031105518341064 train acc 0.721399625935162
epoch 5 batch id 601 loss 1.8957135677337646 train acc 0.7263935108153078
epoch 5 batch id 801 loss 1.3361752033233643 train acc 0.6491495006242197
epoch 5 train acc 0.6476937530305479


  0%|          | 0/104 [00:00<?, ?it/s]

epoch 5 test acc 0.7151932888540031


In [ ]:
# 모델 정보
print(type(model), model, sep="\n\n")

<class '__main__.BERTClassifier'>

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

__모델 저장 - 모델 전체/파라미터만__


In [ ]:
# 상대경로 안됨?
# torch.save(model, './model/sa-tutorial1.pt')
# torch.save(model.state_dict(), './model/sa-tutorial1_StateDict.pt')

torch.save(model, f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt')

In [ ]:
import os

model_path1 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt'
model_path2 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt'

size1 = os.path.getsize(model_path1) / (1024*1024) # mb 단위
size2 = os.path.getsize(model_path2) / (1024*1024) 
print(f"Model size: {size1:.2f} MB")
print(f"Model_StateDict size: {size2:.2f} MB")

Model size: 351.80 MB
Model_StateDict size: 351.77 MB


- __테스트__

In [ ]:
emos = ['기쁨','불안','당황', '슬픔','분노','상처']
def sa_demo(sentence):
    res = predict(sentence)
    print("문장에서 "+ emos[res] + "이(가) 느껴집니다.")

In [ ]:
s1 = "슬슬 종강할 때가 되지 않았나?"
sa_demo(s1)

문장에서 불안이(가) 느껴집니다.


In [ ]:
s2 = "여름이였다"
sa_demo(s2)

문장에서 슬픔이(가) 느껴집니다.


In [ ]:
s3 = "개 빡이네"
sa_demo(s3)

문장에서 당황이(가) 느껴집니다.


In [ ]:
s4 = "지난 달에 산 코인이 많이 올랐어"
sa_demo(s4)

문장에서 기쁨이(가) 느껴집니다.


In [ ]:
s5 = "석범이는 농담을 이어갔다"
sa_demo(s5)

문장에서 분노이(가) 느껴집니다.
